<img src="images/demo-arch.png">

In [ ]:
# Login to Openshift cluster - ammend this according to the UI 
oc login --token=<token> --server=<api-server-url>

# Login to Vault
export VAULT_ADDR=<vault address>
export VAULT_TOKEN=<insert token here>
vault login -method=token token=$VAULT_TOKEN

### Installing the Vault Secrets Operator 
VSO act as a bridge between HashiCorp Vault and Kubernetes, synchronizing secrets from Vault to native Kubernetes Secrets. It manages the full secret lifecycle, including fetching static, dynamic, and PKI-based secrets, and making them available to Kubernetes applications


In [ ]:
helm list -n vault-secrets-operator
oc get pods -n vault-secrets-operator

### Configuring Kubernetes Authentication 

For applications in Kubernetes to authenticate to Vault, we create a Kubernetes type of auth method. 

1. Configure the hostname of the apiServer. 
2. Certificate 
3. A service account JWT used to access the TokenReview API to validate other JWTs during login.

#### Roles and Policies
Tell you what your applications can do in Vault. 
Role maps a serviceAccount to a policy in Vault. 

In [ ]:
# Configuring the Authentication to vault
export KUBE_EXT_API="https://api.68b1524bd4aed54704f69cc0.ap1.techzone.ibm.com:6443"
export KUBENAMESPACE=vault-secrets-operator
export KUBESVCACCOUNTTOKEN=vaultauth-sa-token
export KUBESVCACCOUNT=vaultauth-sa
# Name of the K8s service account token used for verification when Vault connects to minikube for K8s JWT auth
# This is required as Vault is external from the K8s
export KUBESVCACCOUNTTOKEN=vaultauth-sa-token

kubectl create -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: $KUBESVCACCOUNT
  namespace: $KUBENAMESPACE
EOF

kubectl create -f - <<EOF
apiVersion: v1
kind: Secret
metadata:
  name: $KUBESVCACCOUNTTOKEN
  namespace: $KUBENAMESPACE
  annotations:
    kubernetes.io/service-account.name: $KUBESVCACCOUNT
type: kubernetes.io/service-account-token
EOF

kubectl create -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: role-tokenreview-binding
  namespace: $KUBENAMESPACE
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: system:auth-delegator
subjects:
  - kind: ServiceAccount
    name: $KUBESVCACCOUNT
    namespace: $KUBENAMESPACE
EOF

# Enable the Kubernetes auth method 
export K8SAUTHPATH=openshift
vault auth disable $K8SAUTHPATH
vault auth enable -path $K8SAUTHPATH kubernetes 
export JWT_TOKEN_DEFAULT_DEMONS=$(kubectl get secret -n $KUBENAMESPACE $KUBESVCACCOUNTTOKEN --output='go-template={{ .data.token }}' | base64 --decode)
export KUBE_CA_CERT=$(kubectl get cm -n default config-trusted-cabundle -o jsonpath='{.data.ca-bundle\.crt}')

vault write auth/$K8SAUTHPATH/config \
    token_reviewer_jwt="$JWT_TOKEN_DEFAULT_DEMONS" \
    kubernetes_host="$KUBE_EXT_API" \
    kubernetes_ca_cert="$KUBE_CA_CERT"

vault read auth/$K8SAUTHPATH/config

# Role name to be used by openshift to read dynamic secrets
# export K8SROLE=kv-auth-role-openshift

# vault write auth/$K8SAUTHPATH/login role="$K8SROLE" jwt=$JWT_TOKEN_DEFAULT_DEMONS


In [ ]:
# Setting up policy for KV secrets engine

vault policy write policy-kv-readonly - <<EOF
# Read actual secret data
path "kv-vso-demo/data/*" {
  capabilities = ["read", "list", "subscribe"]
  subscribe_event_types = ["*"]
}

# List secrets (optional, useful for discovery)
path "kv-vso-demo/metadata/*" {
  capabilities = ["list", "read"]
}

path "kv-vso-demo/*" {
    capabilities = ["read", "list", "subscribe"]
    subscribe_event_types = ["*"]
}

path "kv*" {
    capabilities = ["list"]
}

path "sys/events/subscribe/*" {
    capabilities = ["read"]
}
EOF

In [ ]:
# Creating role for KV to access secrets
export KUBEAPPSVCACCOUNT=app
export KUBEAPPNAMESPACE=default

kubectl create -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: app
  labels:
    app: php
EOF


kubectl create -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app: php
  name: php
spec:
  replicas: 1
  selector:
    matchLabels:
      app: php
  strategy: {}
  template:
    metadata:
      labels:
        app: php
    spec:
      serviceAccountName: app
      containers:
      - image: quay.io/colintkn/vault-php-app:latest
        name: php
EOF

kubectl create -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  labels:
    app: php
  name: php
spec:
  ports:
  - port: 80
    protocol: TCP
    targetPort: 80
  selector:
    app: php
  type: NodePort
EOF


vault write auth/$K8SAUTHPATH/role/$K8SROLE \
   bound_service_account_names=$KUBEAPPSVCACCOUNT \
   bound_service_account_namespaces=$KUBEAPPNAMESPACE \
   period=60 \
   token_policies=policy-kv-readonly


### Enabling the Secrets Engine
Enable the secret engine and create a sample secret

In [ ]:
export KVPATH=kv-vso-demo
vault secrets disable $KVPATH
vault secrets enable -path $KVPATH -version=2 kv

vault kv put kv-vso-demo/app-secret \
    username="user123" \
    password="Passw0rd" \
    apiKey="12345678"

### VaultConnection 
`VaultConnection` is a custom resource is used by the Vault Secrets Operator (VSO) to define how to connect to a specific Vault cluster. It allows the operator to manage secrets from different Vault environments by providing the necessary details, such as the address and authentication method, for VSO to connect to that Vault cluster and synchronize secrets with Kubernetes. 


In [ ]:
kubectl create -f - <<EOF
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultConnection
metadata:
  namespace: vault-secrets-operator
  name: vault-connection
spec:
  # required configuration
  # address to the Vault server.
  address: http://162.133.140.29:8200
  skipTLSVerify: true
EOF

kubectl describe VaultConnection vault-connection

## VaultAuth 

In [ ]:
echo "K8s namespace: $KUBENAMESPACE"
echo "K8s Auth Path: $K8SAUTHPATH"
echo "K8s role: $K8SROLE"
echo "K8s Service Account: $KUBESVCACCOUNT"

kubectl create -f - <<EOF
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultAuth
metadata:
  name: vault-auth-kv-app
  namespace: default
spec:
  kubernetes:
    audiences:
    - vault
    role: kv-auth-role-openshift
    serviceAccount: app-with-secret
    tokenExpirationSeconds: 600
  method: kubernetes
  mount: openshift
  vaultConnectionRef: vault-connection
EOF 

In [ ]:
kubectl describe VaultAuth vault-auth-kv-app 

## VaultStaticSecret

In [ ]:
kubectl create -f - <<EOF
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  namespace: $KUBEAPPNAMESPACE
  name: vault-static-secret
spec:
  vaultAuthRef: vault-auth-kv-app
  mount: kv-vso-demo
  type: kv-v2
  path: app-secret
  namespace: root
  refreshAfter: 1s
  destination:
    create: true
    name: kv-app-secret
  syncConfig:
    instantUpdates: true
EOF

In [ ]:
kubectl describe VaultStaticSecret vault-static-secret

In [ ]:
kubectl get secret kv-app-secret -o json 

In [ ]:
kubectl describe deploy php-with-secret

## Vault PKI Engine


In [ ]:
# Adapted from https://developer.hashicorp.com/vault/tutorials/pki/pki-engine?productSlug=vault&tutorialSlug=secrets-management&tutorialSlug=pki-engine&variants=vault-deploy%3Aselfhosted
vault secrets enable -path=pki pki
vault secrets tune -max-lease-ttl=8760h pki

vault write -field=certificate pki/root/generate/internal \
     common_name="example.com" \
     issuer_name="root-2023" \
     ttl=87600h > root_2023_ca.crt

vault list pki/issuers/

vault read pki/issuer/$(vault list -format=json pki/issuers/ | jq -r '.[]') \
 | tail -n 6





Create a role for the root CA. Creating this role allows for specifying an issuer when necessary for the purposes of this scenario. This also provides a simple way to transition from one issuer to another by referring to it by name.



In [ ]:
vault write pki/roles/2023-servers allow_any_name=true

Configure the CA and CRL URLs.



In [ ]:
vault write pki/config/urls \
     issuing_certificates="$VAULT_ADDR/v1/pki/ca" \
     crl_distribution_points="$VAULT_ADDR/v1/pki/crl"

### Generate Intermediate CA

In [ ]:
# enable the pki secrets engine at the pki_int path
vault secrets enable -path=pki_int pki

# Tune the pki_int secrets engine to issue certificates with a maximum time-to-live (TTL) of 43800 hours
vault secrets tune -max-lease-ttl=43800h pki_int

Execute the following command to generate an intermediate and save the CSR as pki_intermediate.csr. This command will produce no output.

In [ ]:
vault write -format=json pki_int/intermediate/generate/internal \
     common_name="example.com Intermediate Authority" \
     issuer_name="example-dot-com-intermediate" \
     | jq -r '.data.csr' > pki_intermediate.csr

Sign the intermediate certificate with the root CA private key, and save the generated certificate as intermediate.cert.pem. This command produces no output. 

In [ ]:
vault write -format=json pki/root/sign-intermediate \
     format=pem_bundle ttl="43800h" \
     issuer_ref="root-2023" \
     csr=@pki_intermediate.csr \
     | jq -r '.data.certificate' > intermediate.cert.pem

After signing the CSR and the root CA returns a certificate, it can be imported back into Vault.

In [ ]:
vault write pki_int/intermediate/set-signed certificate=@intermediate.cert.pem

# Create a role 
A role is a logical name that maps to a policy used to generate those credentials. It allows configuration parameters to control certificate common names, alternate names, the key uses that they are valid for, and more.



In [ ]:
# Create a role named example-dot-com which allows subdomains, and specify the default issuer ref ID as the value of issuer_ref.
vault write pki_int/roles/example-dot-com \
     issuer_ref="$(vault read -field=default pki_int/config/issuers)" \
     allowed_domains="example.com" \
     allow_subdomains=true \
     max_ttl="720h"

In [ ]:
export PKI_SERVICE_ACCOUNT=vault-pki-sa

vault policy write policy-pki-demo - <<EOF
path "pki_int/issue/example-dot-com" {
  capabilities = ["update"]
}
EOF

kubectl create -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: $PKI_SERVICE_ACCOUNT
  namespace: $PKI_NS
  labels:
    app: $PKI_SERVICE_ACCOUNT
EOF 

vault write auth/openshift/role/pki-role \
    bound_service_account_names=$PKI_SERVICE_ACCOUNT \
    bound_service_account_namespaces=default \
    policies=policy-pki-demo \
    ttl=1h

In [ ]:
kubectl create -f - <<EOF
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultAuth
metadata:
  name: vault-auth-pki
spec:
  kubernetes:
    audiences:
    - vault
    role: pki-role
    serviceAccount: $PKI_SERVICE_ACCOUNT
    tokenExpirationSeconds: 600
  method: kubernetes
  mount: openshift
  vaultConnectionRef: vault-connection
EOF 

kubectl describe VaultAuth vault-auth-pki


In [ ]:

kubectl create -f - <<EOF
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultPKISecret
metadata:
  name: pki-cert
spec:
  vaultAuthRef: vault-auth-pki
  mount: pki_int
  role: example-dot-com
  commonName: php-app.example.com
  ttl: 1m
  destination: 
    create: true
    type: kubernetes.io/tls
    name: pki-cert-secret
    overwrite: true
EOF

kubectl describe VaultPKISecret pki-cert


In [ ]:
oc describe secret pki-cert-secret

In [ ]:
# Create deployment, service, ingress
kubectl create -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app: php-with-cert
  name: php-with-cert
spec:
  replicas: 1
  selector:
    matchLabels:
      app: php-with-cert
  template:
    metadata:
      labels:
        app: php-with-cert
    spec:
      serviceAccountName: php-with-cert
      containers:
      - image: quay.io/colintkn/vault-php-cert-app:latest
        name: php
EOF

kubectl create -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  labels:
    app: php-with-cert
  name: php-with-cert
spec:
  ports:
  - port: 80
    protocol: TCP
    targetPort: 80
  selector:
    app: php-with-cert
  type: NodePort
EOF

kubectl create -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: php-with-cert
  labels:
    app: php-with-cert
EOF

kubectl create -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: php-with-cert-ingress
  annotations:
    ingressClassName: "openshift-default"   # or "haproxy" / your ingress controller class
spec:
  tls:
    - hosts:
        - php-app.example.com   # 🔹 replace with your real hostname
      secretName: pki-cert-secret
  rules:
    - host: php-app.example.com # 🔹 same hostname as above
      http:
        paths:
          - path: /
            pathType: Prefix
            backend:
              service:
                name: php-with-cert
                port:
                  number: 80A
EOF


In [ ]:
oc describe ingress php-with-cert-ingress 

In [ ]:
oc describe svc php-with-cert


In [ ]:

oc describe deploy php-with-cert

In [ ]:
# Add the following to the host file, x.x.x.x being the load balancer ip:
vi /etc/hosts
x.x.x.x php-app.example.com

# VSO CSI 
Vault Secrets Operator CSI allows you to provide secrets directly to Kubernetes pods as protected secrets.
https://developer.hashicorp.com/vault/docs/deploy/kubernetes/vso/csi

In [ ]:
# Create a policy for csi driver to fetch secrets:

vault policy write policy-csi-kv-readonly - <<EOF
path "kv-vso-demo/data/*" {
    capabilities = ["read"]
}

path "sys/license/status" {
    capabilities = ["read"]
}
EOF

In [ ]:
oc get ds -n vault-secrets-operator


In [ ]:
kubectl create -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: app-with-csi-secret
  namespace: default
EOF

vault write auth/openshift/role/csi \
   bound_service_account_names=app-with-csi-secret \
   bound_service_account_namespaces=default \
   period=60 \
   token_policies=policy-csi-kv-readonly

# https://developer.hashicorp.com/vault/docs/deploy/kubernetes/vso/api-reference#csisecrets

In [ ]:
kubectl create -f - <<EOF
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultAuth
metadata:
  name: vault-auth-csi
  namespace: default
spec:
  kubernetes:
    audiences:
    - vault
    role: csi
    serviceAccount: app-with-csi-secret
    tokenExpirationSeconds: 600
  method: kubernetes
  mount: openshift
  vaultConnectionRef: vault-connection
EOF

kubectl describe VaultAuth vault-auth-csi



In [ ]:
# https://developer.hashicorp.com/vault/docs/deploy/kubernetes/vso/api-reference#csisecrets

kubectl create -f - <<EOF
apiVersion: secrets.hashicorp.com/v1beta1
kind: CSISecrets
metadata:
  name: app-csi-secret
  namespace: default
spec:
  vaultAuthRef:
    name: vault-auth-csi
  secrets:
    vaultStaticSecrets:
      - mount: kv-vso-demo
        path: app-secret
        type: kv-v2
  accessControl:
    serviceAccountPattern: "app-with-csi-secret"
    namespacePatterns:
      - "default"
    podNamePatterns:
      - "^php-with-"
  syncConfig:
    containerState:
      namePattern: "^(php)$"
EOF

kubectl describe CSISecrets app-csi-secret

In [ ]:
kubectl create -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app: php-with-csi
  name: php-with-csi
spec:
  replicas: 1
  selector:
    matchLabels:
      app: php-with-csi
  template:
    metadata:
      labels:
        app: php-with-csi
    spec:
      serviceAccountName: app-with-csi-secret
      containers:
      - image: quay.io/colintkn/vault-php-app:latest
        name: php
        volumeMounts:
        - name: csi-secrets
          mountPath: /vault/secrets
      volumes:
      - name: csi-secrets
        csi:
          driver: csi.vso.hashicorp.com
          volumeAttributes:
            csiSecretsName: app-csi-secret
            csiSecretsNamespace: default
EOF

kubectl get deploy php-with-csi -o yaml

kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  labels:
    app: php-with-csi
  name: php-with-csi
spec:
  ports:
  - port: 80
    protocol: TCP
    targetPort: 80
  selector:
    app: php-with-csi
  type: NodePort
EOF

In [ ]:
oc exec -it deploy/php-with-csi -- cat /vault/secrets/static_secret_0_username
oc exec -it deploy/php-with-csi -- cat /vault/secrets/static_secret_0_password
oc exec -it deploy/php-with-csi -- cat /vault/secrets/static_secret_0_apiKey

# Vault Kubernetes Engine
The Vault Kubernetes secrets engine is for securely generating and managing short-lived, dynamic Kubernetes service account tokens for applications, which are automatically revoked after their lease expires. 
https://developer.hashicorp.com/vault/docs/secrets/kubernetes

In [ ]:
kubectl apply -f - <<EOF
apiVersion: v1
kind: Namespace
metadata:
  name: vault
EOF

### Configuring the ServiceAccount
Vault connects to Kubernetes using its own service account. It's necessary to ensure that the service account Vault uses will have permissions to manage service account tokens, and optionally manage service accounts, roles, and role bindings. These permissions can be managed using a Kubernetes role or cluster role. The role is attached to the Vault service account with a role binding or cluster role binding.

In [ ]:
kubectl apply -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: vault
  namespace: vault
EOF


kubectl describe sa vault -n vault

In [ ]:
kubectl apply -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: k8s-full-secrets-abilities-with-labels
rules:
- apiGroups: [""]
  resources: ["namespaces"]
  verbs: ["get"]
- apiGroups: [""]
  resources: ["serviceaccounts", "serviceaccounts/token"]
  verbs: ["create", "update", "delete"]
- apiGroups: ["rbac.authorization.k8s.io"]
  resources: ["rolebindings", "clusterrolebindings"]
  verbs: ["create", "update", "delete"]
- apiGroups: ["rbac.authorization.k8s.io"]
  resources: ["roles", "clusterroles"]
  verbs: ["bind", "escalate", "create", "update", "delete"]
EOF

kubectl apply -f - <<EOF 
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: vault-token-creator-binding
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: k8s-full-secrets-abilities-with-labels
subjects:
- kind: ServiceAccount
  name: vault
  namespace: vault
EOF

kubectl create -f - <<EOF
apiVersion: v1
kind: Secret
metadata:
  name: vault-service-account-token
  namespace: vault
  annotations:
    kubernetes.io/service-account.name: vault
type: kubernetes.io/service-account-token
EOF

Create a role binding to bind the role to Vault's service account and grant Vault permission to manage tokens.



In [ ]:
kubectl describe clusterrole k8s-full-secrets-abilities-with-labels -n vault
kubectl describe clusterrolebinding vault-token-creator-binding -n vault

Create a JWT token for the Vault serviceAccount

In [ ]:
oc describe secret vault-service-account-token -n vault

Configure the Kubernetes secret engine

In [ ]:
vault secrets enable kubernetes

export JWT_TOKEN_DEFAULT_DEMONS=$(kubectl get secret -n vault vault-service-account-token --output='go-template={{ .data.token }}' | base64 --decode)
export KUBE_CA_CERT=$(kubectl get cm -n default config-trusted-cabundle -o jsonpath='{.data.ca-bundle\.crt}')
export KUBE_EXT_API="https://api.68b1524bd4aed54704f69cc0.ap1.techzone.ibm.com:6443"

vault write kubernetes/config \
    service_account_jwt="$JWT_TOKEN_DEFAULT_DEMONS" \
    kubernetes_host="$KUBE_EXT_API" \
    kubernetes_ca_cert="$KUBE_CA_CERT"

Create a test serviceAccount with the role and roleBindings 

In [ ]:
# Create test service account
kubectl create namespace test

kubectl apply -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: test-service-account-with-generated-token
  namespace: test
EOF

kubectl apply -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: test-role-list-pods
  namespace: test
rules:
- apiGroups: [""]
  resources: ["pods"]
  verbs: ["list"]
EOF

kubectl apply -f  - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: test-role-abilities
  namespace: test
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: test-role-list-pods
subjects:
- kind: ServiceAccount
  name: test-service-account-with-generated-token
  namespace: test
EOF

vault write kubernetes/roles/my-role \
    allowed_kubernetes_namespaces="*" \
    service_account_name="test-service-account-with-generated-token" \
    token_default_ttl="10m"

kubectl describe serviceaccount test-service-account-with-generated-token -n test

In [ ]:
kubectl describe role test-role-list-pods -n test

In [ ]:
kubectl describe rolebinding test-role-abilities -n test

Create a Vault role on the Kubernetes Secrets engine 

In [ ]:
vault write kubernetes/roles/my-role \
    allowed_kubernetes_namespaces="*" \
    service_account_name="test-service-account-with-generated-token" \
    token_default_ttl="10m"

Generate a service account token 

In [ ]:
vault write kubernetes/creds/my-role \
    kubernetes_namespace=test
    
    

In [ ]:
export SA_TOKEN=$(vault write -field=service_account_token kubernetes/creds/my-role kubernetes_namespace=test) 
echo $SA_TOKEN

Test an API call using the service account token to do a view of the pods in the test namespace

In [ ]:
curl -sk $(kubectl config view --minify -o 'jsonpath={.clusters[].cluster.server}')/api/v1/namespaces/test/pods\
    --header "Authorization: Bearer $SA_TOKEN"